# Ejercicios de evaluación de criptosistemas basados en curvas elípticas 
Autora: Blanca Cano Camarero 

## Ejercicio 
Sea $F_{32} = F_2[\xi]_{\xi^5 + \xi^2 +1}$. 
Con la curva elíptica $E(A,B)$ y punto $Q$: 


In [36]:
755577392 % 32

16

$$E(A,B) = E(\xi^4 + \xi^2 + \xi + 1,  \xi^4 + \xi^3 + \xi + 1)$$

$$Q = (1, \xi^4 + \xi^3 + \xi^2 + \xi + 1)$$

In [37]:
F.<xi> = GF(2^5, modulus = GF(2)[x](x^5 + x^2 + 1))
E = EllipticCurve(F, [1,(xi^4 + xi^2 + xi + 1), 0, 0, (xi^4 + xi^3 + xi + 1)])
Q = E.point((1, xi^4 + xi^3 + xi^2 + xi + 1))
display(E)
display(Q)

Elliptic Curve defined by y^2 + x*y = x^3 + (xi^4+xi^2+xi+1)*x^2 + (xi^4+xi^3+xi+1) over Finite Field in xi of size 2^5

(1 : xi^4 + xi^3 + xi^2 + xi + 1 : 1)

## Apartado 1
Calcula mediante el algoritmo de Shank o mediante el algoritmo 9, $\log_Q \mathcal O$.

In [38]:
o = E.point(0)
o

(0 : 1 : 0)

En virtud del teorema de Hasse calculamos una cota del orden: 

$$|E| \leq q + 1 + \lfloor 2 \sqrt{q} \rfloor$$

In [39]:
q = 32
cota_orden = q + 1 + floor(2 * sqrt(q))
print(f"La cota es {cota_orden}")

La cota es 44


Tomamos: 

In [40]:
f = ceil(sqrt(
    q + 1 + floor(2 * sqrt(q))
))
f

7

Y construimos la tabla: 

In [41]:
tabla = [[i, i*Q] for i in range(0, f)]
tabla

[[0, (0 : 1 : 0)],
 [1, (1 : xi^4 + xi^3 + xi^2 + xi + 1 : 1)],
 [2, (xi^4 + xi^3 + xi : xi^4 + xi^3 + xi^2 + xi + 1 : 1)],
 [3, (xi^3 + xi^2 : xi^4 + xi + 1 : 1)],
 [4, (xi^3 + xi : xi^4 + xi^3 : 1)],
 [5, (xi^4 + 1 : xi^3 : 1)],
 [6, (xi^3 + xi^2 + xi : xi^3 + xi^2 : 1)]]

Calculamos ahora

$$Q^{-f} = -f Q$$

In [42]:
Q_elevado_menos_f = -f*Q
Q_elevado_menos_f

(xi^2 + xi + 1 : xi + 1 : 1)

Vamos a construir funciones auxiliares: 
1. Un elemento pertenece a la tabla
2. Índice j-esimo de la tabla

In [43]:
def EstaEnLaTabla(h, tabla):
    return len([x for x in tabla if x[1] == h]) > 0

def SacaIndice(h, tabla):
    incidencias = [x for x in tabla if x[1] == h] 
    return incidencias[0][0]

# Cálculo del logaritmo 

h = o
indice_i = 0 
indice_j = 0

for i in range(1, len(tabla)):
    h = h + (Q_elevado_menos_f)
    
    print("h_"+ str(i) + " = " + str(h))
    
    if EstaEnLaTabla(h, tabla):
        indice_i = i
        indice_j = SacaIndice(h, tabla)
        print("Sí que está en la tabla (i = " + str(indice_i) + ", j = " + str(indice_j) + ")")
        break

# Por lo tanto el índice es 

logaritmo = indice_j + indice_i * f
print(f"El logaritmo es {logaritmo}")

h_1 = (xi^2 + xi + 1 : xi + 1 : 1)
h_2 = (xi^3 + xi^2 : xi^4 + xi + 1 : 1)
Sí que está en la tabla (i = 2, j = 3)
El logaritmo es 17


In [44]:
# Comprobación de la correción 
logaritmo * Q

(0 : 1 : 0)

## Apartado 2

Generar clave par de claves pública y privadas para el algoritmo de ECDH

### Solución 

Sea $E$ nuestra curva, de forma que $|E| = hn$, con $n$ primo,  $h$ pequeño y $\|Q\| = n$: 

In [50]:
# Vemos parámetros 
print(f"Orden E= {E.order()}")
print(f"Orden de Q={Q.order()}")

# Tomamos entonces 
n = 17
h = 2

Orden E= 34
Orden de Q=17


Alice toma número aleatorio entre 
$$2 \leq a \leq n-1$$ 
y hace público: 
$$(E, Q, aQ)$$

In [53]:
a = ZZ.random_element(2,17)

print(f"a = {a}")

a = 5


Alice envia a Bob $P_A=aQ$

In [56]:
P_A = a*Q
P_A

(xi^4 + 1 : xi^3 : 1)

Bob por su parte hace lo mismo: 
generara $P_B = bQ$ y se la envía a Alice 

In [58]:
b = ZZ.random_element(2, n)
print(f"b = {b}")
P_B = b * Q
print(f"P_B = {P_B}")

b = 7
P_B = (xi^2 + xi + 1 : xi^2 : 1)


Veamos que las respectivas claves compartidas son 
La de Alice a(P_B) y la Bob b(P_A):

In [59]:
k_alice = a * P_B
k_bob = b * P_A
print("Clave de Alice:")
display(k_alice)
print("\nClave de Bob:")
display(k_bob)

Clave de Alice:


(1 : xi^4 + xi^3 + xi^2 + xi + 1 : 1)


Clave de Bob:


(1 : xi^4 + xi^3 + xi^2 + xi + 1 : 1)

Notemos que ambas satisfacen que: 

In [61]:
a * (P_B) == b * (P_A)

True

### Apartado 3

Cifra el mensaje 

$$(\xi^3 + \xi^2 + 1, \xi^4 + \xi^2) \in F^{2}_{32}$$
mediante el criptosistema de Menezes-Vanston. 



### Solución 

Como en el protocolo la generación de claves se mantiene idéntica mantendremos las del apartado anterior.

In [66]:
# Mensaje 

ms = [xi^3 + xi^2 + 1, xi^4 + xi^2]


Supongamos que es Bob el que quiere enviar el mensaje, hacemos la partición: 
$$(x_0, y_0) = b(aQ)$$

In [63]:
x_0 = (b*(P_A))[0]
y_0 = (b*(P_A))[1]

x_0, y_0

(1, xi^4 + xi^3 + xi^2 + xi + 1)

El cifrado del mensaje particionado viene dado por: 
$$E(M)= E(m_1, m_2) = bQ, M+b(P_A) = (b Q, x_0m_1, y_0m_2)$$

In [68]:
x_0m_1 = x_0 * ms[0]
y_0m_2 = y_0 * ms[1]

criptograma = [b*Q, x_0m_1, y_0m_2]
print("El criptograma es: ")
display(criptograma)

El criptograma es: 


[(xi^2 + xi + 1 : xi^2 : 1), xi^3 + xi^2 + 1, xi^4 + xi^2 + 1]

### Apartado 4 

Descifra el mensaje anterior

### Solución 

Supongamos que es Alice la que quiere descifrar el mensaje, pare ello calcula primero:
$$a(C_1) = a(P_B) = (x_0, y_0)$$

In [69]:
C_1 = criptograma[0]
aC_1 = a * C_1

x_0 = aC_1[0]
y_0 = aC_1[1]

x_0, y_0

(1, xi^4 + xi^3 + xi^2 + xi + 1)

Para desencriptarlo debe hacer: 
$$D(C_1, c_2, c_3) = (x^{-1}c_2, y_0^{-1}c_3)$$

In [75]:
descifrado = [
    x_0^-1 * criptograma[1], 
    y_0^-1 * criptograma[2]
]
print(f"El mensaje descifrado es:\n {descifrado}")
descifrado == ms

El mensaje descifrado es:
 [xi^3 + xi^2 + 1, xi^4 + xi^2]


True